In [1]:
import pandas as pd
from google.colab import files

# Load datasets
climate_data = pd.read_csv("Climate clean data.csv")
cpi_data = pd.ExcelFile("Final cleaned CPI Data.xlsx")
main_dataset = pd.read_csv("Final_Cleaned_Main_Dataset_Midpoint_Estimation.csv")
financial_stress_data = pd.read_csv("Financial_Stress_Ratios.csv")
google_trends_data = pd.ExcelFile("Transformed_Cleaned_Google_Trends_Data.xlsx")
unemployment_data = pd.ExcelFile("Unemployment rate.xlsx")

# Read sheets from Excel files
cpi_df = cpi_data.parse("Sheet 1 - Final cleaned CPI Dat")
google_trends_df = google_trends_data.parse("Sheet 1 - Transformed_Google_Tr")
unemployment_df = unemployment_data.parse("Sheet 1 - Unemployment rate")

# Standardize column names
climate_data.rename(columns={"Year": "YEAR", "Quarter": "QUARTER"}, inplace=True)
financial_stress_data.rename(columns={"Year": "YEAR", "Quarter": "QUARTER"}, inplace=True)
main_dataset.rename(columns={"Year": "YEAR", "Quarter": "QUARTER"}, inplace=True)
google_trends_df.rename(columns={"Year": "YEAR", "Quarter": "QUARTER"}, inplace=True)

# Fix Province Name Mismatches
province_corrections = {
    "Newfoundland&labrador": "Newfoundland and Labrador",
    "Prince Edward": "Prince Edward Island",
    "Québec": "Quebec"
}

datasets_to_clean = [climate_data, cpi_df, main_dataset, financial_stress_data, google_trends_df, unemployment_df]

for dataset in datasets_to_clean:
    dataset["Province"] = dataset["Province"].replace(province_corrections).str.strip()

# Merge Datasets
merged_df = climate_data.merge(cpi_df, on=["Province", "YEAR", "QUARTER"], how="outer")
merged_df = merged_df.merge(main_dataset, on=["Province", "YEAR", "QUARTER"], how="outer")
merged_df = merged_df.merge(financial_stress_data, on=["Province", "YEAR", "QUARTER"], how="outer")
merged_df = merged_df.merge(google_trends_df, on=["Province", "YEAR", "QUARTER"], how="outer")
merged_df = merged_df.merge(unemployment_df, on=["Province", "YEAR", "QUARTER"], how="outer")

# Filter Only Required Provinces & Quarters
valid_provinces = [
    "Alberta", "British Columbia", "Manitoba", "New Brunswick",
    "Newfoundland and Labrador", "Nova Scotia", "Ontario",
    "Prince Edward Island", "Quebec", "Saskatchewan"
]

valid_quarters = [
    (2021, 2), (2021, 3), (2021, 4),
    (2022, 1), (2022, 2), (2022, 3), (2022, 4),
    (2023, 1), (2023, 2), (2023, 3)
]

filtered_df = merged_df[
    (merged_df["Province"].isin(valid_provinces)) &
    (merged_df[["YEAR", "QUARTER"]].apply(tuple, axis=1).isin(valid_quarters))
]

# Save and Download Cleaned Dataset
filtered_df.to_csv("Final_Filtered_Dataset.csv", index=False)
files.download("Final_Filtered_Dataset.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>